### Data Analysis using Pyspark

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [2]:
!pip install pyspark

     -------------------------------------- 310.8/310.8 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 200.5/200.5 kB 1.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317146 sha256=92b4426d62cf27bdfd427f1ab1a3c24f9f809235b6b0461431cec2845d9f0e43
  Stored in directory: c:\users\julia\appdata\local\pip\cache\wheels\9f\34\a4\159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max , struct
import matplotlib.pyplot as plts

In [4]:
spark = SparkSession.builder.appName('spark_app').getOrCreate()

RuntimeError: Java gateway process exited before sending its port number

In [5]:
listening_csv_path = '/content/drive/MyDrive/dataset/listenings.csv'
listening_df = spark.read.format('csv').option('inferSchema', True).option('header', True).load(listening_csv_path)

NameError: name 'spark' is not defined

In [6]:
listening_df.show()

NameError: name 'listening_df' is not defined

In [ ]:
listening_df = listening_df.drop('date')

In [ ]:
listening_df = listening_df.na.drop()

In [ ]:
listening_df.printSchema()

In [ ]:
shape = (listening_df.count(), len(listening_df.columns)
print(shape)

In [8]:
q0 = listening_df.select('artist', 'track')
q0.show()

NameError: name 'listening_df' is not defined

Let's find all of the records of those users who have listened to Rihanna

In [ ]:
q1 = listening_df.select('*').filter(listening_df.artist == 'Rihanna')
q1.show()

Let's find top 10 users who are fan of Rihanna

In [ ]:
q2 = listening_df.select('user_id').filter(listening_df.artist == 'Rihanna').groupby('user_id').agg(count('user_id').alias('count')).orderBy(desc('count')).limit(10)
q2.show()

find top 10 famous tracks

In [ ]:
q3 = listening_df('artist', 'track').groupby('artist', 'track').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q3.show()

find top 10 famous tracks of Rihanna

In [ ]:
q4 = listening_df.select('artist', 'track').filter(listening_df.artist == 'Rihanna').groupby('artist', 'track').agg(count('*').alias('count')).orderBy('count').limit(10)
q4.show()

find top 10 famous albums

In [ ]:
q5 = listening_df.select('artist','album').groupby('artist', 'album').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q5.show()

importing the genre.csv file:

In [ ]:
genre_csv_path = '/content/drive/MyDrive/dataset/genre.csv'
genre_df = spark.read.format('csv').option('inferSchema', True).option('header', True).load(genre_csv_path)

Let's inner join these two data frames

In [ ]:
data = listening_df.join(genre_df, how = 'inner', on = ['artist'])
data.show()

fintop 10 users who a fan of pop music

In [ ]:
q6 = data.select('user_id').filter(data.genre == 'pop').groupby('user_id').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q6.show()

fin top 10 famous genres

In [ ]:
q7 = data.select('genre').groupby('genre').agg(count'*')).orderBy(desc('count')).limit(10)
q7.show()

find out each user favorite genre

In [ ]:
q8_1 = data.select('user_id', 'genre').groupby('user_id', 'genre').agg(count('*').alias('count').orderBy(desc('user_id'))
q8_1.show()                                                                                                              

In [ ]:
q8_2 = q8_1.groupby('user_id').agg(max(struct(col('count'), col('genre'))).alias('max')).select(col('user_id'), col('max.genre'))
q8_2.show

found out how many pop, rock, metal and hip hop singers we have and then visualize it using bar chart

In [ ]:
q9 = genre_df.select('genre').filter(  (col('genre')=='pop') | (col('genre')=='rock') | (col('genre')=='metal') | (col('genre')=='hip hop') ).groupby('genre').agg(count('genre')).alias('count')
q9.show()

Now let's visualize the results using matplotlib

In [ ]:
q9_list = q9.collect()
labels = [row['genre'] for row in q9_list]
counts = [row['count'] for row in q9_list]

In [ ]:
plts.bar(labels, counts)